# **NEURAL NETWORK USING ADAM OPTIMIZER**
by **Jean Carlo Codogno** - 09/10/2017

<br/>

+ **INTRODUCTION**
+ **ADAM OPTIMIZER**


## INTRODUCTION

The propose this notebook is change the optimizer of this notebook (https://www.kaggle.com/jcodogno/neural-network-using-sgd-0-9897), changing the Stochastic Gradient Descent for Adam Optimizer.

**For computional reason was used a little epochs, in my tests I used 30 epochs and I got 0.99071 in accuracy on public test**


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import sys

seed = 782
np.random.seed(seed)

In [ ]:
df = pd.read_csv("../input/train.csv")
train = df.as_matrix()

train_y = train[:,0].astype('int8')
train_x = train[:,1:].astype('float64')

train = None

print("Shape Train Images: (%d,%d)" % train_x.shape)
print("Shape Labels: (%d)" % train_y.shape)

In [ ]:
df = pd.read_csv("../input/test.csv")
test = df.as_matrix().astype('float64')
print("Shape Test Images: (%d,%d)" % train_x.shape)

In [ ]:
train_x /=255
test /= 255

print("Min: %.2f" % np.min(train_x))
print("Max: %.2f" % np.max(train_x))

In [ ]:
train_y = pd.get_dummies(train_y).as_matrix()
print(train_y[0])

In [ ]:
def ReLu(x, derivative=False):
    if(derivative==False):
        return x*(x > 0)
    else:
        return 1*(x > 0)

x = np.arange(20)-10
relu = ReLu(x)

plt.plot(x, relu)
plt.show()

In [ ]:
def Softmax(x):
    x -= np.max(x)
    sm = (np.exp(x).T / np.sum(np.exp(x),axis=1)).T
    return sm

x = np.arange(20)-10
softmax = Softmax([x])

plt.plot(x, softmax[0])
plt.show()

In [ ]:
def CreateWeights():
    ##Initialization of the Weights and the Biases with the random gaussian function with mean zeron, and variance between 1/sqtr(num_inputs_layer)
    
    ninputs = 784
    wl1 = 500 ##Number of neurons in the first layer
    wl2 = 300 ##Number of neurons in the second layer
    nclass = 10 ##Numer of the class, in this case it is the number of the digits.
    
    #layer1
    w1 = np.random.normal(0, ninputs**-0.5, [ninputs,wl1])
    b1 = np.random.normal(0, ninputs**-0.5, [1,wl1])
    
    #Layer2
    w2 = np.random.normal(0, wl1**-0.5, [wl1,wl2])
    b2 = np.random.normal(0, wl1**-0.5, [1,wl2])

    #Layer3
    w3 = np.random.normal(0, wl2**-0.5, [wl2,nclass])
    b3 = np.random.normal(0, wl2**-0.5, [1,nclass])
    
    return [w1,w2,w3,b1,b2,b3]

In [ ]:
def Dropout(x, keep_prop):
    mask = np.random.binomial([np.ones_like(x)],(1-keep_prop))[0]  / (1-keep_prop)
    return x*mask

In [ ]:
def predict(weights, x, keep_prop=0):
    
    w1,w2,w3,b1,b2,b3  = weights 
    
    #1-Hidden Layer
    first = ReLu(x@w1+b1)
    first = Dropout(first, keep_prop)

    #2-Hidden Layer
    second = ReLu(first@w2+b2)
    second = Dropout(second, keep_prop)
    
    #Output Layer
    return [first, second, Softmax(second@w3+b3)]

In [ ]:
def accuracy(output, y):
    hit = 0
    output = np.argmax(output, axis=1)
    y = np.argmax(y, axis=1)
    for y in zip(output, y):
        if(y[0]==y[1]):
            hit += 1

    p = (hit*100)/output.shape[0]
    return p

In [ ]:
def log2(x):
    if(x!=0):
        return np.log(x)
    else:
        return -np.inf
    
def log(y):
    return [[log2(nx) for nx in x]for x in y]

def cost(Y_predict, Y_right):
    
    Loss = -np.mean((np.nan_to_num(Y_right*log(Y_predict)) + np.nan_to_num((1-Y_right)*log(1-Y_predict))),keepdims=True)
    return Loss

In [ ]:
porcent_valid = 0.1
VALID_SIZE = round(train_x.shape[0]*porcent_valid)

index_data = np.arange(train_x.shape[0])
np.random.shuffle(index_data)

x_train = train_x[index_data[VALID_SIZE:]]
x_valid = train_x[index_data[:VALID_SIZE]]


d_train = train_y[index_data[VALID_SIZE:]]
d_valid = train_y[index_data[:VALID_SIZE]]

train_x = None
train_y = None

x_train.shape

## ADAM OPTIMIZER

Adam optimizer was propose by Diederik P. Kingma, Jimmy Ba in *"Adam: A Method for Stochastic Optimization"*, 2015, the simplified weight update is:


$$ m = beta1.m + (1-beta1).dw $$
$$ mt = \dfrac{m} {(1-beta1^t)} $$
$$ v = beta2.v + (1-beta2).(dw^2) $$
$$ vt = \dfrac{v}{(1-beta2^t)}$$
$$ w  -= \eta . \dfrac{m}{ (\sqrt{v} + \epsilon)} $$

Where dw is the gradient. In the paper "Adam: A Method for Stochastic Optimization"* recommended values are ϵ = 1e-8, beta1 = 0.9, beta2 = 0.999.

In the code need just some changes.


Reference: http://cs231n.github.io/neural-networks-3/




In [ ]:
def ADAM(weights, x, t, outputs, eta, beta1, beta2, eps, i, nabla, cache=None):
    
    w1,w2,w3,b1,b2,b3  = weights
    
    
    if(cache==None):
            vw1 = np.zeros_like(w1)
            mw1 = np.zeros_like(w1)
            
            vw2 = np.zeros_like(w2)
            mw2 = np.zeros_like(w2)
            
            vw3 = np.zeros_like(w3)
            mw3 = np.zeros_like(w3)
            
            vb1 = np.zeros_like(b1)
            mb1 = np.zeros_like(b1)
            
            vb2 = np.zeros_like(b2)
            mb2 = np.zeros_like(b2)
            
            vb3 = np.zeros_like(b3)
            mb3 = np.zeros_like(b3)
    else:
        vw1,mw1,vw2,mw2,vw3,mw3,vb1,mb1,vb2,mb2,vb3,mb3 = cache
    
    first, second, y = outputs
   
    w3_delta = (t-y)/x.shape[0]
    
    w2_error = w3_delta@w3.T

    w2_delta = w2_error * ReLu(second,derivative=True)

    w1_error = w2_delta@w2.T
    w1_delta = w1_error * ReLu(first,derivative=True)
    
    
    dw3 = (second.T@w3_delta + nabla*2*w3)
    mw3 = beta1*mw3 + (1-beta1)*dw3
    mt = (mw3) / (1-beta1**i)
    vw3 = beta2*vw3 + (1-beta2)*(dw3**2)
    vt = (vw3) / (1-beta2**i)
    w3 += eta * mt/(np.sqrt(vt) + eps)
    
    db3 = (w3_delta.sum(axis=0)+ nabla*2*b3)
    mb3 = beta1*mb3 + (1-beta1)*db3
    mt = (mb3) / (1-beta1**i)
    vb3 = beta2*vb3 + (1-beta2)*(db3**2)
    vt = (vb3) / (1-beta2**i)
    b3 += eta * mt/(np.sqrt(vt) + eps)
    
    dw2 = (first.T@w2_delta + nabla*2*w2)
    mw2 = beta1*mw2 + (1-beta1)*dw2
    mt = (mw2) / (1-beta1**i)
    vw2 = beta2*vw2 + (1-beta2)*(dw2**2)
    vt = (vw2) / (1-beta2**i)
    w2 += eta * mt/(np.sqrt(vt) + eps)
    
    db2 = (w2_delta.sum(axis=0) + nabla*2*b2)
    mb2 = beta1*mb2 + (1-beta1)*db2
    mt = (mb2) / (1-beta1**i)
    vb2 = beta2*vb2 + (1-beta2)*(db2**2)
    vt = (vb2) / (1-beta2**i)
    b2 += eta * mt/(np.sqrt(vt) + eps)
    
    dw1 = (x.T@w1_delta + nabla*2*w1)
    mw1 = beta1*mw1 + (1-beta1)*dw1
    mt = (mw1) / (1-beta1**i)
    vw1 = beta2*vw1 + (1-beta2)*(dw1**2)
    vt = (vw1) / (1-beta2**i)
    w1 += eta * mt/(np.sqrt(vt) + eps)
    
    db1 = (w1_delta.sum(axis=0) + nabla*2*b1)
    mb1 = beta1*mb1 + (1-beta1)*db1
    mt = (mb1) / (1-beta1**i)
    vb1 = beta2*vb1 + (1-beta2)*(db1**2)
    vt = (vb1) / (1-beta2**i)
    b1 += eta * mt/(np.sqrt(vt) + eps)
    
    
    weights = [w1,w2,w3,b1,b2,b3]
    cache = [vw1,mw1,vw2,mw2,vw3,mw3,vb1,mb1,vb2,mb2,vb3,mb3]
    
    return weights, cache

In [ ]:
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter

def elastic_transform(image, alpha, sigma, random_state=None):
    """Elastic deformation of images as described in [Simard2003]_.
    .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
       Convolutional Neural Networks applied to Visual Document Analysis", in
       Proc. of the International Conference on Document Analysis and
       Recognition, 2003.
    """
    if random_state is None:
        random_state = np.random.RandomState(None)

    shape = image.shape
    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha

    x, y = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]))
    indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1))

    return map_coordinates(image, indices, order=1).reshape(shape)

In [ ]:
x_t = np.array([elastic_transform(xx.reshape(28,28),15,3).reshape(784) for xx in x_train[0:10]])
plt.subplot(1,2,1)
plt.imshow(x_t[0].reshape(28,28))
plt.subplot(1,2,2)
plt.imshow(x_train[0].reshape(28,28))
plt.show()

In [ ]:
def run(weights, x_train, y_train, x_valid, y_valid, epochs = 10, nbatchs=25, alpha = 1e-3, decay = 0, beta1=0.9, beta2=0.999, eps=1e-8, l2 = 0.001, keep_prop = 0):
    
    pross = x_train.shape[0]*0.05
    
    history = [[],[]]
    
    index = np.arange(x_train.shape[0])
    cache = None
    print("Train data: %d" % (x_train.shape[0]))
    print("Validation data: %d \n" % (x_valid.shape[0]))
    mtime = 0
    inter_adam = 1
    
    for j in range(epochs):
        np.random.shuffle(index)
        t = 0
        iterations = round(x_train.shape[0]/nbatchs)
        prog = ""
        sacurr = 0
        sloss = 0
        sys.stdout.write("\nEpochs: %2d \ %2d \n"% (j+1,epochs))
        stime = 0
        timeIT = time.time()
        for i in range(iterations):
            timeI = time.time()
            f = i*nbatchs
            l = f+nbatchs
            
            if(l>(x_train.shape[0]-1)):
                l = x_train.shape[0]
                
            x = np.array([elastic_transform(xx.reshape(28,28),15,3).reshape(784) for xx in x_train[index[f:l]]])
            y = y_train[index[f:l]]

            outputs = predict(weights, x, keep_prop)
            
            loss = cost(outputs[-1], y)+ l2 * (np.mean(weights[0],keepdims=True)**2+np.mean(weights[1],keepdims=True)**2+np.mean(weights[2],keepdims=True)**2)/3
            
            
            accuracy_t = accuracy(outputs[-1], y)
            
            sacurr += accuracy_t
            sloss += loss
            
            accuracy_train = sacurr/(i+1)
            loss_train = sloss/(i+1)
            
            weights, cache = ADAM(weights, x, y, outputs, alpha, beta1, beta2, eps, inter_adam, l2, cache)
            
            t+= x.shape[0]
            
            qtd = round(t/pross)
            prog = "["
            for p in range(20):
                if(p<qtd-1):
                    prog += "="
                elif(p==qtd-1):
                    prog += ">"
                else:
                    prog += "."
            prog += "]"

            
            stime += time.time()-timeI
            mtime = stime/(i+1)
            mTimeT = mtime * (iterations-i-1)
            
            sys.stdout.write("\r%5d/%5d %s ETA: %3d s - loss: %.4f  acc: %.4f" % (t, x_train.shape[0], prog, mTimeT, loss_train, accuracy_train))
            
            history[0].append([loss_train, accuracy_train])
            inter_adam += 1
        mtime = time.time()-timeIT
        alpha = alpha - (alpha*decay)
        
        outputs = predict(weights, x_valid)
        
        loss_valid = cost(outputs[-1], y_valid)+ l2 * (np.mean(weights[0],keepdims=True)**2+np.mean(weights[1],keepdims=True)**2+np.mean(weights[2],keepdims=True)**2)/3
        accuracy_valid = accuracy(outputs[-1], y_valid)
        
        sys.stdout.write("\r%5d/%5d %s ETA: %3d s loss: %.4f  acc: %.4f - lossValid: %.4f  accValid: %.4f " % ( t, x_train.shape[0], prog, mtime, loss_train, accuracy_train, loss_valid, accuracy_valid))
        history[1].append([loss_valid, accuracy_valid])
        
    return weights, history

In [ ]:
weights = CreateWeights()

alpha = 1e-3
epochs = 18
nbatchs = 100
weights, history = run(weights, 
              x_train, d_train, 
              x_valid, d_valid, 
              epochs = epochs,
              nbatchs=nbatchs, 
              alpha = alpha, 
              decay = 0.1, 
              beta1=0.9, 
              beta2=0.999,
              eps=1e-8,
              l2 = 1e-7, 
              keep_prop = 0.25)

In [ ]:
train_history = np.array(history[0])
t_loss = train_history[:,:1]
t_acc = train_history[:,1:2]

In [ ]:
valid_history = np.array(history[1])
train_history.shape
v_loss = valid_history[:,:1]
v_acc = valid_history[:,1:2]

In [ ]:
plt.figure(figsize=(12,10))

plt.subplot(2, 1, 1)
x = np.arange(epochs)*int(x_train.shape[0]/nbatchs)
plt.plot(x,v_acc)
plt.plot(t_acc)

plt.subplot(2, 1, 2)
plt.plot(x,v_loss)
plt.plot(t_loss)

plt.show()

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, cm[i, j],
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
outputs = predict(weights, x_valid)

In [ ]:
p = np.argmax(outputs[-1],axis=1)
predict_dummies = pd.get_dummies(p).as_matrix().astype('int8')

cm = np.zeros((10,10)).astype(np.int64)

d_valid_int = np.argmax(d_valid, axis=1)

for i in range(predict_dummies.shape[0]):
    cm[d_valid_int[i]] += predict_dummies[i]

print(cm)

In [ ]:
plot_confusion_matrix(cm, range(10))

In [ ]:
s_cm = np.sum(cm,axis=0)
for i in range(10):
    p = cm[i][i]/s_cm[i]
    print("%d - %.3f %%" % (i,p))
    

In [ ]:
w1,w2,w3,b1,b2,b3  = weights

In [ ]:
%matplotlib inline
plt.figure(figsize=(12,10))

y, x = 4,10
for i in range(0,(y*x)):
    ni = np.random.randint(0,w1.shape[1],1)[0]
    plt.subplot(y, x, i+1)
    plt.imshow(w1[:,ni].reshape((28,28)))

plt.show()

In [ ]:
%matplotlib inline
plt.figure(figsize=(12,10))

y, x = 4,10
for i in range(0,(y*x)):
    ni = np.random.randint(0,w2.shape[1],1)[0]
    plt.subplot(y, x, i+1)
    plt.imshow(w2[:,ni].reshape((20,25)))

plt.show()

In [ ]:
%matplotlib inline
plt.figure(figsize=(12,10))

y, x = 1,10
for i in range(0,(y*x)):
    plt.subplot(y, x, i+1)
    plt.imshow(w3[:,i].reshape((20,15)))

plt.show()

In [ ]:
outputs = predict(weights, test)


In [ ]:
d = np.argmax(outputs[-1],axis=1)

In [ ]:


def show_image(image, shape, label="", cmp=None):
    img = np.reshape(image,shape)
    plt.imshow(img,cmap=cmp, interpolation='none')
    plt.title(label)
    



In [ ]:
%matplotlib inline
plt.figure(figsize=(20,12))

y, x = 5,11
for i in range(0,(y*x)):
    plt.subplot(y, x, i+1)
    ni = np.random.randint(0,test.shape[0],1)[0]
    v = str(d[ni])
    show_image(test[ni],(28,28), v, cmp="gray")
plt.show()

In [ ]:
#ImageId,Label

pd.DataFrame({"ImageId": list(range(1,len(d)+1)), "Label": d}).to_csv('output.csv', index=False, header=True)

## REFERENCES

http://cs231n.github.io/neural-networks-3/